# Challenge Tests

## Setup and Helper Functions

In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
from fau_colors import cmaps, register_fausans_font

from pepbench.algorithms.ecg import (
    QPeakExtractionVanLien2013,
)
from pepbench.algorithms.heartbeat_segmentation import HeartbeatSegmentationNeurokit
from pepbench.algorithms.icg import (
    BPointExtractionDrost2022,
)
from pepbench.algorithms.outlier_correction import (
    OutlierCorrectionDummy,
)
from pepbench.datasets import EmpkinsDataset, TimeWindowIcgDataset, ReBeatIcgDataset
from pepbench.evaluation import PepEvaluationChallenge
from pepbench.pipelines import PepExtractionPipeline
from pepbench.plotting import plot_signals, plot_signals_with_reference_labels, plot_signals_with_reference_pep, plot_signals_from_challenge_results

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

In [ ]:
root_path = Path()

In [ ]:
deploy_type = "local"

config_dict = json.load(root_path.joinpath("config.json").open(encoding="utf-8"))

empkins_base_path = Path(config_dict[deploy_type]["empkins_path"])
rebeat_icg_base_path = Path(config_dict[deploy_type]["rebeat_icg_path"])
time_window_icg_path = Path(config_dict[deploy_type]["time_window_icg_path"])

print(time_window_icg_path)

In [ ]:
dataset_empkins = EmpkinsDataset(empkins_base_path, use_cache=True, only_labeled=True)
dataset_empkins

In [ ]:
dataset_rebeat_icg = ReBeatIcgDataset(rebeat_icg_base_path, use_cache=True, only_labeled=True, exclude_annotation_errors=True)
dataset_rebeat_icg

In [ ]:
dataset_time_window_icg = TimeWindowIcgDataset(time_window_icg_path, use_cache=True, only_labeled=True)
dataset_time_window_icg

## Initialize Algorithms

### Heartbeat Segmentation

In [ ]:
heartbeat_segmentation_algo = HeartbeatSegmentationNeurokit()

### ECG - Q-Wave Onset

In [ ]:
q_peak_algo = QPeakExtractionVanLien2013(time_interval_ms=32)

### ICG - B-Point

In [ ]:
b_point_algo = BPointExtractionDrost2022()

### Outlier Correction

In [ ]:
outlier_correction_algo = OutlierCorrectionDummy()

In [ ]:
subset_empkins = dataset_empkins.get_subset(participant=["VP_001", "VP_002"])

In [ ]:
subset_rebeat_icg = dataset_rebeat_icg#.get_subset(participant=["01", "02"])

In [ ]:
subset_time_window_icg = dataset_time_window_icg.get_subset(participant=["IDN_01", "IDN_02"])

In [ ]:
pep_challenge = PepEvaluationChallenge(dataset=subset_time_window_icg, validate_kwargs={"n_jobs": -1})
pipeline = PepExtractionPipeline(
    heartbeat_segmentation_algo=heartbeat_segmentation_algo,
    q_peak_algo=q_peak_algo,
    b_point_algo=b_point_algo,
    outlier_correction_algo=outlier_correction_algo,
    handle_negative_pep="nan",
    handle_missing_events="ignore",
)
pep_challenge.run(pipeline)
pep_challenge.results_as_df();

In [ ]:
pep_challenge.results_agg_total_

In [ ]:
pep_challenge.results_agg_mean_std_

In [ ]:
pep_challenge.results_per_sample_[["absolute_error_per_sample_ms"]].groupby(["participant", "phase"]).agg("mean").unstack()

In [ ]:
subset_id = ("IDN_01", "Baseline")
#subset_analysis = dataset_rebeat_icg.get_subset(participant=subset_id[0], phase=subset_id[1], label_period=subset_id[2])
subset_analysis = dataset_time_window_icg.get_subset(participant=subset_id[0], phase=subset_id[1])

In [ ]:
plot_signals_from_challenge_results(subset_analysis, pep_challenge.results_per_sample_.xs(subset_id));

In [ ]:
pep_challenge.results_per_sample_["error_per_sample_ms"].abs().idxmax()

In [ ]:
dataset_rebeat_icg = ReBeatIcgDataset(rebeat_icg_base_path, use_cache=True, only_labeled=False, return_clean=True)

In [ ]:
subset_rebeat_icg = dataset_rebeat_icg.get_subset(participant="23", phase="CognitiveWorkload")#, label_period=3)

In [ ]:
from pepbench.datasets.rebeat_icg._helper import _load_b_point_annotations

In [ ]:
b_points = _load_b_point_annotations("/Users/richer/Documents/PhD/Projects/EmpkinS/HealthPsychology_D03/Data/2024_08_PEP_Benchmarking/ReBeatICG_Dataset/03_ExpertAnnotations/Annotat_Subject_23_task_CW.mat")

In [ ]:
fig, ax = plt.subplots()

data = subset_rebeat_icg.icg#.iloc[63452:66469]
data.plot(ax=ax)

ax.vlines(
    data.index[subset_rebeat_icg.heartbeats["start_sample"]], 
    ymin=0, ymax=1, 
    transform=ax.get_xaxis_transform(),
    color="grey", ls="--"
)

ax.vlines(
    data.index[[63452, 66469]], 
    ymin=0, ymax=1, 
    transform=ax.get_xaxis_transform(),
    color="red"
)

ax.vlines(
    data.index[b_points["sample_relative"] - 70], 
    ymin=0, ymax=1, 
    transform=ax.get_xaxis_transform(),
    color="cyan"
)